In [2]:
!pip3 install ultralytics diffusers transformers accelerate scipy safetensors deap schedule

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.1/749.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [3]:
import random
import math
import sys
import hashlib
import numpy
import copy
import pandas as pd
from deap import creator, base, tools, algorithms
import torch
from scipy.spatial.distance import cosine
import requests
from PIL import Image
from statistics import mean
from ultralytics import YOLO
import time

import logging
from bs4 import BeautifulSoup
import requests
import schedule
from diffusers import (
    StableDiffusionPipeline,
    EulerDiscreteScheduler,
    StableDiffusionImg2ImgPipeline,
)
import calendar
import time
import numpy as np
from io import BytesIO
import cv2
import random
import argparse


# Optimizer parameters
# numTuples = int(ConfigSectionMap("Optimizer")['numtuples'])


def int_to_binary_and_select_elements(integer, element_list):
    binary_representation = bin(integer)[2:]
    selected_elements = []
    for i, digit in enumerate(binary_representation):
        if digit == "1":
            selected_elements.append(element_list[i])
    return selected_elements


# Parameters for the boxes
thickness = 2
fontScale = 0.5

model_id = "stabilityai/stable-diffusion-2"
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(
    model_id, scheduler=scheduler, torch_dtype=torch.float16
)  # for cuda
# pipe = StableDiffusionPipeline.from_pretrained(model_id, scheduler=scheduler, torch_dtype=torch.float32) # for cpu
pipe = pipe.to("cuda")
# pipe = pipe.to("cpu")
model = YOLO("yolov8n.pt")  # load a pretrained YOLOv8n detection model
model.train(data="coco128.yaml", epochs=3)  # train the model
colors = np.random.randint(0, 255, size=(len(model.names), 3), dtype="uint8")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


scheduler/scheduler_config.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

        3/3      5.25G      1.212      1.453       1.24        221        640: 100%|██████████| 8/8 [00:01<00:00,  5.92it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  4.84it/s]

                   all        128        929      0.646      0.534      0.625      0.465



3 epochs completed in 0.005 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.1.42 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:04<00:00,  1.24s/it]


                   all        128        929      0.647      0.541      0.626      0.467
                person        128        254      0.763      0.673      0.768      0.543
               bicycle        128          6      0.481      0.333      0.324       0.27
                   car        128         46      0.686      0.217      0.286      0.175
            motorcycle        128          5      0.696      0.922      0.938      0.742
              airplane        128          6      0.684      0.667      0.913      0.717
                   bus        128          7      0.652      0.714      0.735      0.675
                 train        128          3      0.539      0.667      0.806      0.662
                 truck        128         12          1      0.326      0.493      0.276
                  boat        128          6      0.286      0.167       0.36      0.243
         traffic light        128         14      0.746      0.211      0.206       0.14
             stop sig

In [11]:
def read_box(box):
    cords = box.xyxy[0].tolist()
    cords = [round(x) for x in cords]
    class_id = model.names[box.cls[0].item()]
    conf = round(box.conf[0].item(), 2)
    return [class_id, cords, conf]


def addBoxesImage(currentImage, boxesInfo):
    image = cv2.imread(currentImage)
    for box in boxesInfo:
        class_id = box[0]
        confidence = box[2]
        color = [int(c) for c in colors[list(model.names.values()).index(class_id)]]
        #        color = colors[list(model.names.values()).index(class_id)]
        cv2.rectangle(
            image,
            (box[1][0], box[1][1]),
            (box[1][2], box[1][3]),
            color=color,
            thickness=thickness,
        )
        text = f"{class_id}: {confidence:.2f}"
        (text_width, text_height) = cv2.getTextSize(
            text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=fontScale, thickness=thickness
        )[0]
        text_offset_x = box[1][0]
        text_offset_y = box[1][1] - 5
        box_coords = (
            (text_offset_x, text_offset_y),
            (text_offset_x + text_width + 2, text_offset_y - text_height),
        )
        overlay = image.copy()
        cv2.rectangle(
            overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED
        )
        image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
        cv2.putText(
            image,
            text,
            (box[1][0], box[1][1] - 5),
            cv2.FONT_HERSHEY_SIMPLEX,
            fontScale=fontScale,
            color=(0, 0, 0),
            thickness=thickness,
        )
    cv2.imwrite(currentImage + "_yolo8.png", image)


def createNegativePrompt(selection):
    items = [
        "illustration",
        "painting",
        "drawing",
        "art",
        "sketch",
        "lowres",
        "error",
        "cropped",
        "worst quality",
        "low quality",
        "jpeg artifacts",
        "out of frame",
        "watermark",
        "signature",
    ]
    # integer_input =  random.randint(0,2**len(fixed_length_list)-1)
    if selection > 2 ** len(items) - 1:
        selection %= 2 ** len(items) - 1
    selected_elements = int_to_binary_and_select_elements(selection, items)
    return ", ".join(selected_elements)


def createPosPrompt(prompt, selection):
    items = [
        "photograph",
        "digital",
        "color",
        "Ultra Real",
        "film grain",
        "Kodak portra 800",
        "Depth of field 100mm",
        "overlapping compositions",
        "blended visuals",
        "trending on artstation",
        "award winning",
    ]
    # integer_input =  random.randint(0,2**len(fixed_length_list)-1)
    if selection > 2 ** len(items) - 1:
        selection %= 2 ** len(items) - 1
    selected_elements = int_to_binary_and_select_elements(selection, items)
    return prompt + ", " + ", ".join(selected_elements)


def text2img(prompt, configuration={}):
    num_inference_steps = configuration["num_inference_steps"]
    guidance_scale = configuration["guidance_scale"]
    negative_prompt = createNegativePrompt(configuration["negative_prompt"])
    prompt = createPosPrompt(prompt, configuration["positive_prompt"])
    guidance_rescale = configuration["guidance_rescale"]
    num_images_per_prompt = 4
    seed = configuration["seed"]
    generator = torch.Generator("cuda").manual_seed(seed)
    # generator = torch.Generator("cpu").manual_seed(seed)
    print(prompt)
    print(negative_prompt)

    #Measure GPU time
    starter, ender = torch.cuda.Event(enable_timing=True), torch.cuda.Event(enable_timing=True)
    starter.record()
    imagesAll = pipe(
        prompt,
        guidance_scale=guidance_scale,
        num_inference_steps=num_inference_steps,
        guidance_rescale=guidance_rescale,
        negative_prompt=negative_prompt,
        generator=generator,
        num_images_per_prompt=num_images_per_prompt,
    ).images
    ender.record()
    torch.cuda.synchronize()
    inference_time = starter.elapsed_time(ender)

    print(inference_time)
    print(imagesAll)
    timestamp = calendar.timegm(time.gmtime())
    images = []
    for i, image in enumerate(imagesAll):
        image.save(
            prompt.replace(" ", "_")
            + "."
            + str(timestamp)
            + "."
            + str(i)
            + "."
            + "image.png"
        )
        images.append(
            prompt.replace(" ", "_")
            + "."
            + str(timestamp)
            + "."
            + str(i)
            + "."
            + "image.png"
        )
    return images, inference_time


def img2text(image_path):
    result = model(image_path)  # predict on an image
    boxesInfo = []
    counting = {}
    for box in result[0].boxes:
        currentBox = read_box(box)
        boxesInfo.append(currentBox)
        if currentBox[0] in counting.keys():
            counting[currentBox[0]] += 1
        else:
            counting[currentBox[0]] = 1
    return counting, boxesInfo


class GAOptimizer:

    def __init__(self, options={}, others={}):
        # GA parameters
        self.numGen = int(options["numgen"])
        self.mutProb = float(options["mut_prob"])
        self.crossProb = float(options["cross_prob"])
        self.numSel = int(options["num_sel"])
        self.muSel = int(options["mu_sel"])
        self.lambdaSel = int(options["lambda_sel"])
        self.innerMutProb = float(options["inner_mut_prob"])
        self.populationSize = int(options["population_size"])
        self.tournamentSel = int(options["tournament_sel"])
        self.weights = options["weights"]
        self.prompt = options["prompt"]

        # Individual Initialization parameters
        # self.tsize = int(options['sizetuples'])
        # self.isize = int(options['numtuples'])
        # self.types = list(options["type"+str(i)] for i in range(self.tsize))
        # self.minInt = int(options['minInt'])
        # self.maxInt = int(options['maxInt'])
        # self.minFloat= float(0)
        # self.maxFloat= float(options['noise'])

    def createElem(self):
        param_ranges_dict = {
            "num_inference_steps": random.randint(1, 100),  # Number of denoising steps
            "guidance_scale": 20
            * random.uniform(0, 1),  # Scale for classifier-free guidance
            "negative_prompt": random.randint(1, 2**9),
            "positive_prompt": random.randint(1, 2**14),
            "guidance_rescale": random.uniform(0, 1),
            "num_images_per_prompt": 4,
            "seed": random.randint(1, 2**9),
        }
        return param_ranges_dict

    ## Documentation for randomInit
    variable_name = ""

    def randomInit(self, icls):
        ind = self.createElem()
        # print(ind)
        return icls(self.createElem())

    ## Documentation for randomInit

    def mutUniform(self, individual):
        ind2 = copy.copy(individual)
        mutInd = self.createElem()
        for elem in individual.keys():
            if random.random() < self.innerMutProb:
                ind2[elem] = mutInd[elem]
        return (ind2,)

    def crossOverDict(self, ind1, ind2):
        #        return ind1,ind2
        print("Crossing")
        cutpoint = random.randrange(1, len(ind1.keys()))
        chrom1_list = [(k, v) for k, v in ind1.items()]
        chrom2_list = [(m, n) for m, n in ind2.items()]

        offspring_1 = chrom1_list[1:cutpoint] + chrom2_list[cutpoint : len(chrom2_list)]

        offspring_2 = chrom2_list[1:cutpoint] + chrom1_list[cutpoint : len(chrom1_list)]
        print("Showing the offprint")
        print(offspring_1)
        offspring_1 = dict(offspring_1)
        offspring_2 = dict(offspring_2)
        chrom_offspring_1, chrom_offspring_2 = copy.copy(ind1), copy.copy(ind2)
        # print(chrom1.__dict__)
        # print(offspring_1.__dict__)
        chrom_offspring_1.update(offspring_1)
        chrom_offspring_2.update(offspring_2)
        print("Final offpring")
        print(chrom_offspring_1)
        print("Original")
        print(ind1)
        #        chrom_offspring_1.__dict__ = offspring_1
        #        chrom_offspring_2.__dict__ = offspring_2
        return [chrom_offspring_1, chrom_offspring_2]

    #    def fitness(self,inputs):
    #        return 1

    def optimize(self):
        print("Optimizing...")
        #        creator.create("FitnessMin", base.Fitness, weights=(-1.0,-1.0,-1.0))
        creator.create("FitnessMax", base.Fitness, weights=self.weights)
        creator.create("Individual", dict, fitness=creator.FitnessMax)
        toolbox = base.Toolbox()
        toolbox.register("individual", self.randomInit, creator.Individual)
        toolbox.register("population", tools.initRepeat, list, toolbox.individual)
        toolbox.register("evaluate", self.evalFitness)
        toolbox.register("mate", self.crossOverDict)
        toolbox.register("mutate", self.mutUniform)
        toolbox.register("select", tools.selNSGA2)
        # The statistics for the logbook
        stats = tools.Statistics(key=lambda ind: ind.fitness.values)
        stats.register("avg", numpy.mean, axis=0)
        stats.register("std", numpy.std, axis=0)
        stats.register("min", numpy.min, axis=0)
        stats.register("max", numpy.max, axis=0)

        hof = tools.ParetoFront()

        population = toolbox.population(n=self.populationSize)
        # The genetic algorithm, this implementation ia mu+lambda
        # it is feeded with a population of individuals, a mutation
        # and crossover probabilities and a number of generations
        offspring, logbook = algorithms.eaMuCommaLambda(
            population,
            toolbox,
            mu=self.muSel,
            lambda_=self.lambdaSel,
            cxpb=self.crossProb,
            mutpb=self.mutProb,
            ngen=self.numGen,
            stats=stats,
            halloffame=hof
        )
        # the top ten individuals are printed
        # topTen = tools.selBest(population, k=10)
        # print(topTen)
        best = tools.selBest(population, k=1)
        return best[0], offspring, logbook, hof

    def get_caption_similarity(self, text_a, text_b):
        texts = [text_a, text_b]
        inputs = self.tokenizer(
            texts, padding=True, truncation=True, return_tensors="pt"
        )
        # Get the embeddings
        with torch.no_grad():
            embeddings = self.modelText(
                **inputs, output_hidden_states=True, return_dict=True
            ).pooler_output
        similarity_score = 1 - cosine(embeddings[0], embeddings[1])
        return similarity_score

    def evalFitness(self, individual):
        print("Fitness")
        avgPrecision = 0
        totalCount = 0
        configuration = {
            "num_inference_steps": individual["num_inference_steps"],
            "guidance_scale": individual["guidance_scale"],
            "negative_prompt": individual["negative_prompt"],
            "positive_prompt": individual["positive_prompt"],
            "guidance_rescale": individual["guidance_rescale"],
            "seed": individual["seed"],
        }

        allimages, inference_time = text2img(self.prompt, configuration)
        for currentImage in allimages:
            counting, boxesInfo = img2text(currentImage)
            print(counting)
            addBoxesImage(currentImage, boxesInfo)
            for box in boxesInfo:
                totalCount += 1
                avgPrecision += box[2]

        if avgPrecision == 0:
            image_quality = (0,)
        else:
            image_quality = (
                avgPrecision / totalCount,
            )  # individual['num_inference_steps'],

        # fitness = 0.5 * (image_quality) + 0.5 * (
        #     10 - inference_time
        # )  # we want to maximize the quality while minimize the time, assume the max_time is 10 hours
        print(inference_time)
        return image_quality, inference_time #maximize image_quality and minimize inf_time


In [12]:
prompt = "Two people and a bus"

configuration = {
    "numgen": 5,
    "mut_prob": 0.2,
    "cross_prob": 0.2,
    "num_sel": 10,
    "mu_sel": 5,
    "lambda_sel": 5,
    "inner_mut_prob": 0.2,
    "population_size": 5,
    "tournament_sel": 5,
    "weights": [1, -1],
    "prompt": prompt,
}

print("Loading data")
print("GA")
gen = GAOptimizer(configuration)

sol, offspring, logbook, hof = gen.optimize()
print("Last Generation")
print(offspring)
print("Logs")
print(logbook)
print("Best")
print(sol)
print("Hall of fame")
print(hof)
print("Done")

Loading data
GA
Optimizing...
Fitness
Two people and a bus, photograph, blended visuals, trending on artstation
illustration, sketch, lowres, cropped


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


  0%|          | 0/81 [00:00<?, ?it/s]

104977.6171875
[<PIL.Image.Image image mode=RGB size=768x768 at 0x7BDFF57BF280>, <PIL.Image.Image image mode=RGB size=768x768 at 0x7BDFF57BF8B0>, <PIL.Image.Image image mode=RGB size=768x768 at 0x7BDFF57BD7E0>, <PIL.Image.Image image mode=RGB size=768x768 at 0x7BDFF57BF670>]

image 1/1 /content/Two_people_and_a_bus,_photograph,_blended_visuals,_trending_on_artstation.1712087245.0.image.png: 640x640 2 persons, 9.3ms
Speed: 2.7ms preprocess, 9.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
{'person': 2}

image 1/1 /content/Two_people_and_a_bus,_photograph,_blended_visuals,_trending_on_artstation.1712087245.1.image.png: 640x640 1 person, 1 bus, 8.9ms
Speed: 2.8ms preprocess, 8.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
{'bus': 1, 'person': 1}

image 1/1 /content/Two_people_and_a_bus,_photograph,_blended_visuals,_trending_on_artstation.1712087245.2.image.png: 640x640 5 persons, 1 bus, 1 traffic light, 8.8ms
Speed: 2.5ms preprocess, 8.8ms infe

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/92 [00:00<?, ?it/s]

117896.21875
[<PIL.Image.Image image mode=RGB size=768x768 at 0x7BE017E150F0>, <PIL.Image.Image image mode=RGB size=768x768 at 0x7BE017E15270>, <PIL.Image.Image image mode=RGB size=768x768 at 0x7BE017E149D0>, <PIL.Image.Image image mode=RGB size=768x768 at 0x7BE017E16E90>]

image 1/1 /content/Two_people_and_a_bus,_photograph,_color,_film_grain,_blended_visuals,_trending_on_artstation,_award_winning.1712087998.0.image.png: 640x640 2 persons, 1 bus, 1 truck, 9.5ms
Speed: 2.9ms preprocess, 9.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
{'person': 2, 'bus': 1, 'truck': 1}

image 1/1 /content/Two_people_and_a_bus,_photograph,_color,_film_grain,_blended_visuals,_trending_on_artstation,_award_winning.1712087998.1.image.png: 640x640 4 persons, 1 bus, 8.7ms
Speed: 2.6ms preprocess, 8.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)
{'person': 4, 'bus': 1}

image 1/1 /content/Two_people_and_a_bus,_photograph,_color,_film_grain,_blended_visuals,_trendin

  0%|          | 0/81 [00:00<?, ?it/s]

104026.0859375
[<PIL.Image.Image image mode=RGB size=768x768 at 0x7BE017E417E0>, <PIL.Image.Image image mode=RGB size=768x768 at 0x7BE017E417B0>, <PIL.Image.Image image mode=RGB size=768x768 at 0x7BE017E418A0>, <PIL.Image.Image image mode=RGB size=768x768 at 0x7BE017E40C40>]

image 1/1 /content/Two_people_and_a_bus,_photograph,_blended_visuals,_trending_on_artstation.1712088103.0.image.png: 640x640 2 persons, 1 elephant, 9.5ms
Speed: 2.7ms preprocess, 9.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
{'person': 2, 'elephant': 1}

image 1/1 /content/Two_people_and_a_bus,_photograph,_blended_visuals,_trending_on_artstation.1712088103.1.image.png: 640x640 3 persons, 1 bus, 2 dogs, 1 skateboard, 9.8ms
Speed: 2.5ms preprocess, 9.8ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
{'person': 3, 'dog': 2, 'skateboard': 1, 'bus': 1}

image 1/1 /content/Two_people_and_a_bus,_photograph,_blended_visuals,_trending_on_artstation.1712088103.2.image.png: 640x640

TypeError: selNSGA2() got an unexpected keyword argument 'tournsize'